In [1]:
import pandas as pd
import numpy as np
from data_preprocessing import compressed_pickle, decompress_pickle
#from fastai.tabular.all import *
import os
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split

In [5]:
# Y_train = decompress_pickle(
#     "./data/partitioned/BikeRental_Y_train.pbz2")
# Y_test = decompress_pickle("./data/partitioned/BikeRental_Y_test.pbz2")
# X_train = decompress_pickle(
#     "./data/partitioned/BikeRental_X_train.pbz2")
# X_test = decompress_pickle("./data/partitioned/BikeRental_X_test.pbz2")

Y_train = pd.read_csv(
    "../data/partitioned/BikeRental_Y_train.csv", index_col=[0])
Y_test = pd.read_csv(
    "../data/partitioned/BikeRental_Y_test.csv", index_col=[0])
X_train = pd.read_csv(
    "../data/partitioned/BikeRental_X_train.csv", index_col=[0])
X_test = pd.read_csv(
    "../data/partitioned/BikeRental_X_test.csv", index_col=[0])

X_train = X_train.drop('datetime', axis=1)
X_test = X_test.drop('datetime', axis=1)

Y_train_mean = Y_train.mean()
#Y_train_meandev = sum((Y_train-Y_train_mean)**2)
Y_train_meandev = ((Y_train-Y_train_mean)**2).sum()
#Y_test_meandev = sum((Y_test-Y_train_mean)**2)
Y_test_meandev = ((Y_test-Y_train_mean)**2).sum()
model = CatBoostRegressor(loss_function='RMSE', depth=6,
                                  learning_rate=0.1, iterations=1000, od_type='Iter', od_wait=10)

try:
    # df = decompress_pickle(
    #     "./data/preprocessed/BikeRental_preprocessed.pbz2")
    df = pd.read_csv(
        "../data/preprocessed/BikeRental_preprocessed.csv", index_col=[0])
    min_max = pd.read_csv("../data/preprocessed/cnt_min_max.csv")
    #min_max = decompress_pickle("./data/preprocessed/cnt_min_max.pbz2")

    model.load_model("../models/catboost/catboost_model")
    print("Model loaded!")
except FileNotFoundError:
    cat_var = ["season", "yr", "mnth", "hr", "holiday",
               "weekday", "workingday", "weathersit"]
    for v in cat_var:
        X_train[v] = X_train[v].astype("int64")
        X_test[v] = X_test[v].astype("int64")

    if not os.path.exists("../catboost"):
        os.makedirs("../catboost")

    os.chdir("../catboost/")

    model = CatBoostRegressor(loss_function='RMSE', depth=6,
                              learning_rate=0.1, iterations=1000, od_type='Iter', od_wait=10)

    model.fit(
        X_train, Y_train,
        use_best_model=True,
        cat_features=["season", "yr", "mnth", "hr",
                      "holiday", "weekday", "workingday", "weathersit"],
        eval_set=(X_test, Y_test),
        verbose=True,
        plot=True
    )

    model.save_model("catboost_model", format="cbm")

test_df = df[round(len(df)*0.8):].copy()
test_df = test_df.drop('datetime', axis=1)
test_df = test_df.rename(columns={"cnt": "cnt_real"})
cat_var = ["season", "yr", "mnth", "hr",
                          "holiday", "weekday", "workingday", "weathersit"]

for v in cat_var:
    test_df[v] = test_df[v].astype("int64")
    
#test_df["weathersit"] = test_df["weathersit"].astype("category")

Model loaded!


In [6]:
model.load_model("../models/catboost/catboost_model")

CatBoostError: c:/program files (x86)/go agent/pipelines/buildmaster/catboost.git/catboost/libs/model/model_import_interface.h:19: Model file doesn't exist: ../models/catboost/catboost_model

In [87]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3509 entries, 14035 to 17543
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    3509 non-null   object 
 1   season      3509 non-null   int32  
 2   yr          3509 non-null   int32  
 3   mnth        3509 non-null   int32  
 4   hr          3509 non-null   int32  
 5   holiday     3509 non-null   int32  
 6   weekday     3509 non-null   int32  
 7   workingday  3509 non-null   int32  
 8   weathersit  3509 non-null   int32  
 9   temp        3509 non-null   float64
 10  hum         3509 non-null   float64
 11  windspeed   3509 non-null   float64
 12  cnt_real    3509 non-null   float64
dtypes: float64(4), int32(8), object(1)
memory usage: 274.1+ KB


In [3]:
model.predict(test_df)

array([0.48295031, 0.35815575, 0.26827056, ..., 0.17952609, 0.10981189,
       0.06189975])

In [21]:
Y_train_pred = model.predict(X_train)


In [26]:
len(Y_train_pred)

14035

In [27]:
len(Y_train)

14035

In [30]:
Y_train["cnt"].array-Y_train_pred

<PandasArray>
[ -0.01740931094899223,  0.026321410493944132,  0.026302026379979228,
  0.007410541234612366, -0.004884540732600751, -0.005697943404561784,
  0.002427901584593591,  0.023002433794956447,  -0.01634522345964342,
  -0.03231921598225554,
 ...
  -0.03493597441004764,  0.027114039036676918, -0.023393094499955802,
 0.0002827263491042009,  -0.02066473921464046,  0.006047232074150921,
  0.014147657708227901,   0.06199214526194652,   0.11055616971667892,
  0.059950546415074246]
Length: 14035, dtype: float64

In [32]:
Y_train_dev = sum((Y_train["cnt"].array-Y_train_pred)**2)
# Y_train_dev = ((Y_train-Y_train_pred)**2).sum()
r2 = 1 - Y_train_dev/Y_train_meandev
print("R2 =", r2)

Y_test_pred = model.predict(X_test)
Y_test_dev = sum((Y_test["cnt"].array-Y_test_pred)**2)
# Y_test_dev = ((Y_test-Y_test_pred)**2).sum()
pseudor2 = 1 - Y_test_dev/Y_test_meandev
print("Pseudo-R2 =", pseudor2)

R2 = cnt    0.949566
dtype: float64
Pseudo-R2 = cnt    0.915737
dtype: float64


In [55]:
r2.values[0]

0.949566284416855

In [34]:
dir(r2)

['T',
 '_AXIS_ALIASES',
 '_AXIS_IALIASES',
 '_AXIS_LEN',
 '_AXIS_NAMES',
 '_AXIS_NUMBERS',
 '_AXIS_ORDERS',
 '_AXIS_REVERSED',
 '_HANDLED_TYPES',
 '__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__array_priority__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__long__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__